In [1]:
!mkdir data
!wget http://www.manythings.org/anki/rus-eng.zip -O 'data/rus-eng.zip'
!wget https://object.pouta.csc.fi/OPUS-OpenSubtitles/v1/moses/en-ru.txt.zip -O 'data/en-ru.txt.zip'

mkdir: data: File exists
--2024-09-12 08:20:27--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16305013 (16M) [application/zip]
Saving to: ‘data/rus-eng.zip’

data/rus-eng.zip    100%[===================>]  15.55M  4.00MB/s    in 5.3s    

2024-09-12 08:20:33 (2.95 MB/s) - ‘data/rus-eng.zip’ saved [16305013/16305013]

--2024-09-12 08:20:33--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v1/moses/en-ru.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 610036 (596K) [application/zip]
Saving to: ‘data/en-ru.txt.zip’

data/en-ru.txt.zip  100%[===================>] 595.74K  1.72MB/s    in 0.3s    

2024-0

In [2]:
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import zipfile
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# dataset_anki_path = 'data/rus-eng.zip'
# with zipfile.ZipFile(dataset_anki_path, 'r') as zip_ref:
#     zip_ref.extractall('anki_data')
# dataset_opensub_path = 'data/en-ru.txt.zip'
# with zipfile.ZipFile(dataset_opensub_path, 'r') as zip_ref:
#     zip_ref.extractall('opensubtitles_data')

In [4]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) \
        if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яёъА-ЯЁЪ.!?]+", r" ", s)
    return s


In [6]:
def load_opensubtitles_data(file_path_ru, file_path_en, num_samples=100_000):
    input_texts = []
    with open(file_path_en, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text = line.strip()
            input_texts.append(normalizeString(en_text))
    target_texts = []
    with open(file_path_ru, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            ru_text = line.strip()
            target_texts.append(normalizeString(ru_text))
    return input_texts, target_texts

def load_anki_data(file_path, num_samples=100_000):
    input_texts = []
    target_texts = []
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines[:num_samples]:
            en_text, ru_text = line.strip().split('\t')[:-1]
            input_texts.append(normalizeString(en_text))
            target_texts.append(normalizeString(ru_text)) 
    return input_texts, target_texts

In [7]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # lines = open('anki_data/rus.txt', encoding='utf-8').\
    #     read().strip().split('\n')

    # pairs = [[normalizeString(s) for s in l.split('\t')][0:2] for l in lines]
    anki_input_texts, anki_target_texts = load_anki_data('anki_data/rus.txt')
    opensub_input_texts, opensub_target_texts = load_opensubtitles_data('opensubtitles_data/OpenSubtitles.en-ru.ru',
                                                                 'opensubtitles_data/OpenSubtitles.en-ru.en')
    pairs = list(zip(anki_input_texts, anki_target_texts)) 
    # pairs = list(zip(opensub_input_texts, opensub_target_texts))
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [8]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH  and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [9]:
MAX_LENGTH = 10


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    pairs = filterPairs(pairs)
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'rus', False)
print(random.choice(pairs))

Reading lines...
Counted words:
eng 2157
rus 4609
('i m shivering .', 'я дрожу .')


In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers=1):
        super(EncoderRNN, self).__init__()
        self.num_layers=num_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size,num_layers)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.rnn(output, hidden)
        return output, hidden

    def initHidden(self):
        if str(self.rnn)[:4] == 'LSTM':
            return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device)
                ,torch.zeros(self.num_layers, 1, self.hidden_size, device=device))
        else:
            return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,num_layers=1):
        super(DecoderRNN, self).__init__()
        self.num_layers=num_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size,num_layers)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        if str(self.rnn)[:4] == 'LSTM':
            return (torch.zeros(self.num_layers, 1, self.hidden_size, device=device)
                ,torch.zeros(self.num_layers, 1, self.hidden_size, device=device))
        else:
            return torch.zeros(self.num_layers, 1, self.hidden_size, device=device)

In [11]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [12]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    # Encode input tensor

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    # Decode using the encoded hidden state
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            _, topi = decoder_output.topk(1)

            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length
    



In [13]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [14]:
def trainIters(encoder, decoder, n_iters, print_every=1000,
               learning_rate=0.01):
    start = time.time()
    print_time = time.time()
    print_iter = 0
    print_loss_total = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
    for iter_ in range(1, n_iters + 1):
        training_pair = training_pairs[iter_ - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        print_iter += 1

        if iter_ % 100 == 0 : 
            if (time.time() - print_time > 30) or iter_ == n_iters:
                print_time = time.time()
                print_loss_avg = print_loss_total / print_iter
                print_iter = 0
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter_ / n_iters),
                                             iter_, iter_ / n_iters * 100, print_loss_avg))
        
            
                
        


In [15]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size,1).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words,1).to(device)
trainIters(encoder1, decoder1, 100_000, print_every=10000)

0m 31s (- 86m 1s) (600 0%) 4.1963
1m 1s (- 84m 33s) (1200 1%) 3.6640
1m 32s (- 83m 54s) (1800 1%) 3.5032
2m 6s (- 82m 18s) (2500 2%) 3.3671
2m 41s (- 81m 18s) (3200 3%) 3.2425
3m 12s (- 81m 7s) (3800 3%) 3.2209
3m 46s (- 80m 9s) (4500 4%) 3.1078
4m 21s (- 79m 27s) (5200 5%) 3.0348
4m 52s (- 79m 8s) (5800 5%) 3.0477
5m 24s (- 79m 5s) (6400 6%) 2.9605
5m 55s (- 78m 39s) (7000 7%) 2.9204
6m 26s (- 78m 17s) (7600 7%) 2.8729
6m 57s (- 77m 53s) (8200 8%) 2.7795
7m 28s (- 77m 27s) (8800 8%) 2.7271
7m 59s (- 77m 1s) (9400 9%) 2.8342
8m 31s (- 76m 43s) (10000 10%) 2.6981
9m 1s (- 76m 8s) (10600 10%) 2.7394
9m 33s (- 75m 50s) (11200 11%) 2.6713
10m 4s (- 75m 17s) (11800 11%) 2.6518
10m 38s (- 75m 9s) (12400 12%) 2.6007
11m 10s (- 74m 47s) (13000 13%) 2.5630
11m 43s (- 74m 31s) (13600 13%) 2.5048
12m 15s (- 74m 4s) (14200 14%) 2.5850
12m 48s (- 73m 43s) (14800 14%) 2.5247
13m 20s (- 73m 18s) (15400 15%) 2.5119
13m 52s (- 72m 50s) (16000 16%) 2.4227
14m 26s (- 72m 34s) (16600 16%) 2.4624
14m 59s (

In [25]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
        decoder_input = torch.tensor([[SOS_token]], device=device)  
        decoder_hidden = encoder_hidden
        decoded_words = []
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            _, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
        return decoded_words
            
def evaluateRandomly(encoder, decoder, l, n=3):
    pairs_ = [pair for pair in pairs if len(pair[0].split()) == l]
    for i in range(n):
        
        
        pair = random.choice(pairs_)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words[1:-1])
        print('<', output_sentence)
        print('')             
            
        
        


In [26]:
# 3 words
evaluateRandomly(encoder1, decoder1, 4)

> they re allies .
= они союзницы .
< союзники .

> i m yours .
= я твоя .
< твоя .

> i m prejudiced .
= я пристрастен .
< пристрастен .



In [27]:
# 6 words
evaluateRandomly(encoder1, decoder1,7)

> i m glad that we won .
= я рад что мы победили .
< рад что мы выиграли .

> i m going to be busy .
= я буду занята .
< буду занята .

> i m sure it ll pass .
= я уверен что это проидет .
< уверен что это уверен .



In [28]:
# 7 words
evaluateRandomly(encoder1, decoder1,8)

> i m glad i m a man .
= я рад что я мужчина .
< рад что я мужчина .

> i m as tall as he is .
= я такои же высокии как и он .
< такои же высокии как и он .

> i m glad i m not tom .
= я рад что я не том .
< рад что я не том .



In [31]:
def get_word_vector(word, encoder, decoder, vocab, device='cpu'):
    input_tensor = torch.tensor([[vocab[word]]], device=device)
    
    encoder_hidden = encoder.initHidden()
    encoder_output, encoder_hidden = encoder(input_tensor[0], encoder_hidden)
    
    return encoder_hidden[0].squeeze(0)

def evaluate_word_pairs(encoder, decoder, vocab, synonym_pairs, antonym_pairs, device='cpu'):
    results = []

    def cosine_similarity_torch(vec1, vec2):
        cos = F.cosine_similarity(vec1, vec2)
        return cos.item()

    for word1, word2 in synonym_pairs:
        vec1 = get_word_vector(word1, encoder, decoder, vocab, device)
        vec2 = get_word_vector(word2, encoder, decoder, vocab, device)
        cosine_sim = cosine_similarity_torch(vec1, vec2)
        results.append([word1, word2, cosine_sim, "Синонимы"])

    for word1, word2 in antonym_pairs:
        vec1 = get_word_vector(word1, encoder, decoder, vocab, device)
        vec2 = get_word_vector(word2, encoder, decoder, vocab, device)
        cosine_sim = cosine_similarity_torch(vec1, vec2)
        results.append([word1, word2, cosine_sim, "Антонимы"])

    df = pd.DataFrame(results, columns=["Слово 1", "Слово 2", "Косинусное расстояние", "Тип"])
    return df


synonym_pairs = [("толстыи", "жирныи")]
antonym_pairs = [("толстыи", "худои")]

df_results = evaluate_word_pairs(encoder1, decoder1, output_lang.word2index, synonym_pairs, antonym_pairs)

print(df_results)


   Слово 1 Слово 2  Косинусное расстояние       Тип
0  толстыи  жирныи              -0.014418  Синонимы
1  толстыи   худои              -0.004130  Антонимы
